In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
from copy import deepcopy
%matplotlib inline

## SKLearn

(почти) каждый класс в SKLearn имеет следующие методы:

In [ ]:
class KNN():
    def __init__(self, n_neighbors=5, p=2, metric='minkowski'):
        <your code>
    
    def fit(self, X_train, y_train):
        <your code>
        
    def predict(self, X_test):
        <your code>
        
    def predict_proba(self, X_test):
        <your code>

(у регрессий нет predict_proba, есть только predict)

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

my_knn = KNN(k=<choose your favourite>)
sklearn_knn = KNeighborsClassifier(k=<choose your favourite>)

## House pricing

https://www.kaggle.com/c/house-prices-advanced-regression-techniques

In [ ]:
from sklearn.datasets import load_diabetes

In [ ]:
diabetes = load_diabetes()
diabetes_X = diabetes.data
diabetes_Y = diabetes.target

In [ ]:
diabetes.DESCR.split('\n')

In [ ]:
X_df = pd.DataFrame(diabetes_X, columns=diabetes.feature_names)
X_df.head()

Посмотрим на корреляции признаков между собой:

In [ ]:
import matplotlib.pyplot as plt
plt.subplots(figsize=(10,10))
sns.heatmap(X_df.corr(), square=True)
plt.show()

### Linear Regression

http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import StandardScaler

In [ ]:
lr = LinearRegression()
np.mean(cross_val_score(lr, X_train_sc, diabetes_Y, scoring='neg_mean_squared_error', cv=5))

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(diabetes_X, diabetes_Y, test_size=0.2)

In [ ]:
lr = LinearRegression().fit(x_train, y_train)
lr.predict(x_test).ravel().astype(int)

In [ ]:
np.array(y_test).ravel()

Посмотрим, какие коэффициенты получились у линейной регрессии:

In [ ]:
lr.fit(x_train, y_train)
print(X_df.columns)
print(lr.coef_)

In [ ]:
importances = np.abs(lr.coef_)
indices = np.argsort(importances)[::-1]
columns_num = len(X_df.columns)

plt.figure(figsize=(20, 8))
plt.title("Feature importances")
plt.bar(range(columns_num), importances[indices],
       color="r",  align="center")
plt.xticks(range(columns_num), X_df.columns[indices])
plt.xlim([-1, columns_num])
plt.xticks(rotation=90)
plt.show()

# To-Do

In [ ]:
<Попробуйте применить Lasso и Ridge регрессии к нашему набору данных>

Посмотрим также на корреляцию признаков:

## Предсказание сердечно-сосудистых заболеваний

https://mlbootcamp.ru/round/12/sandbox/

In [ ]:
train_data = pd.read_csv("train_med.csv", delimiter=';')
train_data.head()

In [ ]:
X_train, Y_train = np.array(train_data.drop("cardio", axis=1).drop("id", axis=1)), np.array(train_data['cardio'])

In [ ]:
from sklearn.ensemble import RandomForestClassifier

rf = <random forest>
<cross-val with accuracy>

In [ ]:
<fit random forest>

importances = rf.feature_importances_
std = np.std([tree.feature_importances_ for tree in rf.estimators_], axis=0)
indices = np.argsort(importances)[::-1]

plt.figure(figsize=(20, 8))
plt.title("Feature importances")
plt.bar(range(X_train.shape[1]), importances[indices],
       color="r", yerr=std[indices], align="center")
plt.xticks(range(X_train.shape[1]), train_data.drop(["cardio", "id"], axis=1).columns[indices])
plt.xlim([-1, X_train.shape[1]])
plt.show()

In [ ]:
from sklearn.grid_search import GridSearchCV

rf = RandomForestClassifier(max_depth=3)
params = {
    'max_depth': [3, 10, 50, None],
    'min_samples_split': [2, 3, 4]
}
gsv = GridSearchCV(estimator=rf, param_grid=params, scoring='accuracy', cv=3, verbose=1)
gsv.fit(X_train, Y_train)

In [ ]:
print(gsv.best_params_, gsv.best_score_)

In [ ]:
gsv.grid_scores_

#### XGBoost

In [ ]:
<do the same for GradientBoostingClassifier and other eatimators> 

### Model Ensembles

Мы можем предсказывать не класс, а вероятности классов:

In [ ]:
train_data = pd.read_csv("train_med.csv", delimiter=';')

X_train, Y_train = np.array(train_data.drop("cardio", axis=1)), np.array(train_data['cardio'])

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X_train, Y_train, test_size=0.15, random_state=37)

In [ ]:
rf = RandomForestClassifier(n_estimators=100).fit(X_train, Y_train)
pred = rf.predict_proba(X_test)

In [ ]:
pred

In [ ]:
estimator1 = <choose your favourite>
estimator2 = <choose your second favourite>

estimator1.fit(X_train, Y_train)
estimator2.fit(X_train, Y_train)

pred1 = estimator1.predict_proba(X_test)[:, 1]
pred2 = estimator2.predict_proba(X_test)[:, 1]

pred = <compute average of all the estimators>